# **1. Importing the libraries**


In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import os

# **2. Dataset loading and preprossesing**

loading as utf8 encoding to change dataset into unicode.

In [2]:
file  = open('1661-0.txt', 'r', encoding = "utf8")

append lines

In [3]:
lines=[]
for i in file:
    lines.append(i)

coverting data into one string

In [4]:
data=""
for i in lines:
    data = ' '.join(lines)

cleaning up the data

In [5]:
 data = data.replace('\n', '').replace('\ufeff', '').replace('“','').replace('”','')

Splitting the entire dataset into each word in order without the presence of special characters

In [6]:
data = data.split()
data = ' '.join(data)

# **3.tokenizing**

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [8]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

assiging perticular text a sequence

In [9]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

# **4.sorting and vocab size **

In [10]:
len(sequence_data)

108958

In [11]:
vocab_size = len(tokenizer.word_index)+1

In [12]:
vocab_size

8624

# **5.genrating next word**

In [13]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [14]:
sequence = np.array(sequence)

In [15]:
sequence

array([[ 142, 4680,    1,  986],
       [4680,    1,  986,    5],
       [   1,  986,    5,  125],
       ...,
       [8623,    4,  347,   81],
       [   4,  347,   81,  345],
       [ 347,   81,  345, 1623]])

In [16]:
X=[]
y=[]

for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

In [17]:
X=np.array(X)
y=np.array(y)

In [18]:
y = to_categorical(y, num_classes=vocab_size)

In [19]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Building the LSTM Model

In [20]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000, return_sequences = True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation="softmax"))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             86240     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8624)              8632624   
                                                                 
Total params: 21,767,864
Trainable params: 21,767,864
Non-trainable params: 0
_________________________________________________________________


training model

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model weights.h5", monitor="loss", verbose = 1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer = Adam(learning_rate = 0.001))
model.fit(X, y, epochs=10, batch_size=264, callbacks=[checkpoint])

Epoch 1/10
 27/413 [>.............................] - ETA: 3:22 - loss: 7.9207

KeyboardInterrupt: 

In [ ]:
model = load_model('model weights.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_word(model, tokenizer, text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds=np.argmax(model.predict(sequence))
    predicted_word = ""
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    print(predicted_word)
    return predicted_word



examples


In [ ]:
predict_word(model, tokenizer, "I should have")

1/1 [==============================] - 1s 632ms/step
a


'a'

In [ ]:
predict_word(model, tokenizer, "remained in our lodgings in Baker Street")

1/1 [==============================] - 1s 717ms/step
and


'and'

In [ ]:
predict_word(model, tokenizer, "The Adventure of the Blue")

1/1 [==============================] - 0s 27ms/step
and


'and'

In [ ]:
predict_word(model, tokenizer, "lighting a cigarette")

1/1 [==============================] - 0s 36ms/step
and


'and'

In [ ]:
predict_word(model, tokenizer, "The Adventure")

1/1 [==============================] - 0s 31ms/step
of


'of'

endless conersion loop

In [ ]:
while True:
    text = input("Enter your line: ")
    if text=="?":
        break
    else:
        text = text.split(" ")
        text = text[-3:]
        predict_word(model, tokenizer, text)

Enter your line: I could not help laughing
1/1 [==============================] - 0s 34ms/step
that
Enter your line: One night—it was
1/1 [==============================] - 0s 18ms/step
a
Enter your line: I had seen little of Holmes
1/1 [==============================] - 0s 19ms/step
boone
Enter your line: My own complete
1/1 [==============================] - 0s 27ms/step
statement


KeyboardInterrupt: ignored